In [1]:
%load_ext autoreload

%autoreload 2

In [3]:
import os
from glob import glob
import subprocess
import shutil
import traceback
import random

import joblib
from concurrent.futures import ProcessPoolExecutor

from backports import tempfile

import matplotlib
matplotlib.rcParams['savefig.dpi'] = 144
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

import psrchive

import pipe

In [4]:
data_location = "/nimrod1/aarchiba/0337+17"
data_location_name = data_location+"/{telescope}/{epoch}"

In [5]:
wsrt_data = "/nimrod1/aarchiba/0337+17/WSRT/"
if not os.path.exists(wsrt_data):
    os.mkdir(wsrt_data)

In [6]:
wsrt_raw_location = "/nimrod1/aarchiba/0337+17/WSRT-raw"
wsrt_obs_glob = wsrt_raw_location+"/*/*0337*/"

class DiscontinuityError(ValueError):
    pass

def process_wsrt_day(d):
    success_file = d+"/success"
    if os.path.exists(success_file):
        return open(success_file,"rt").read().strip()
    with tempfile.TemporaryDirectory() as td:
        for s in glob(d+"/Band*"):
            b = s.split("/")[-1]
            subprocess.check_call(["psradd", "-o",td+"/"+b+".tadd.ar"]+glob(s+"/u*.ar"))
        subprocess.check_call(["psradd", "-o", td+"/raw.ar", "-R"]+glob(td+"/Band*.tadd.ar"))
        shutil.copy(td+"/raw.ar", "raw.ar")
        F = psrchive.Archive_load(td+"/raw.ar")
        l = (F.end_time()-F.start_time()).in_seconds()
        M = "%.1f" % (((F.end_time()+F.start_time())).in_days()/2)
        #T = F.get_nsubint()*F.get_first_Integration().get_duration()
        T = F.integration_length()
        if np.abs(l-T)> 5:
            raise DiscontinuityError("Observation in directory %s appears not to be contiguous: "
                             "total integration time %f but start-to-end time %f"
                             % (d,T,l))
        shutil.copy(td+"/raw.ar", wsrt_data+"wsrt_%s.raw.ar" % M)
        with open(success_file,"wt") as f:
            f.write(M)
        return M

In [10]:
g = list(glob(wsrt_obs_glob))
random.shuffle(g)
s = 0
for d in g:
    print d
    try:
        M = process_wsrt_day(d)
        print "success;", M
        s += 1
    except subprocess.CalledProcessError:
        print "psradd failure"
    except DiscontinuityError:
        traceback.print_exc()
print s, len(g)

/nimrod1/aarchiba/0337+17/WSRT-raw/30Jan2012_grid/J0337+17_0/
success; 55956.7
/nimrod1/aarchiba/0337+17/WSRT-raw/12Dec2012/J0337+1715-1380/
psradd failure
/nimrod1/aarchiba/0337+17/WSRT-raw/24Jan2013/J0337+1715-1380/
psradd failure
/nimrod1/aarchiba/0337+17/WSRT-raw/27Mar2012/J0337+1715-1380/
success; 56013.6
/nimrod1/aarchiba/0337+17/WSRT-raw/08Oct2012/J0337+1715-1380/
success; 56208.2
/nimrod1/aarchiba/0337+17/WSRT-raw/18Jun2012/J0337+1715-1380b/
psradd failure
/nimrod1/aarchiba/0337+17/WSRT-raw/03Jun2012/J0337+1715-1380b/
success; 56081.4
/nimrod1/aarchiba/0337+17/WSRT-raw/03Nov2012/J0337+1715-1380/
success; 56234.8
/nimrod1/aarchiba/0337+17/WSRT-raw/24Jul2012/J0337+1715-1380/
psradd failure
/nimrod1/aarchiba/0337+17/WSRT-raw/30Jan2012_grid/J0337+17_+3/
success; 55956.7
/nimrod1/aarchiba/0337+17/WSRT-raw/02Mar2013/J0337+1715-1380a/
psradd failure
/nimrod1/aarchiba/0337+17/WSRT-raw/29Jun2013/J0337+1715-1380b/
psradd failure
/nimrod1/aarchiba/0337+17/WSRT-raw/11Feb2012/J0337+1715-385

OSError: [Errno 12] Cannot allocate memory

In [11]:
if False:
    # Mystery failures
    g = list(glob(wsrt_obs_glob))
    random.shuffle(g)
    def do_one(d):
        try:
            return process_wsrt_day(d)
        except subprocess.CalledProcessError:
            return "psradd problem with %s" % d
        except DiscontinuityError:
            return "discontinuity problem with %s" % d
    joblib.Parallel(n_jobs=-1)(joblib.delayed(do_one)(d) for d in g)

In [12]:
if False:
    # Mystery failures
    g = list(glob(wsrt_obs_glob))
    random.shuffle(g)
    def do_one(d):
        try:
            return process_wsrt_day(d)
        except subprocess.CalledProcessError:
            return "psradd problem with %s" % d
        except DiscontinuityError:
            return "discontinuity problem with %s" % d
    with ProcessPoolExecutor() as P:
        list(P.map(do_one,g))

Now the challenge is to fix as many of the failed observations as possible